
### This version does not use dask to load imagery


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
#comment test, 15 Sep 2023

In [2]:
from osgeo import gdal, ogr, gdal_array # I/O image data
import numpy as np # math and array handling
import matplotlib.pyplot as plt # plot figures
from sklearn.ensemble import RandomForestClassifier # classifier
import pandas as pd # handling large data as table sheets
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix  # calculating measures for accuracy assessment

import seaborn as sn

import datetime

import joblib

In [3]:
import pickle

In [4]:
import os

# print(os.environ['GDAL_NUM_THREADS'])
os.environ['PROJ_LIB'] = "/work/pi_gstuart_umass_edu/kate/conda/share/proj"

In [5]:
# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# define a number of trees that should be used (default = 500)
est = 500

# how many cores should be used?
# -1 -> all available cores
n_cores = 60

# the remote sensing image you want to classify
img_RS = r'/work/pi_cschweik_umass_edu/Ryan/Modeling_stack/Mid_High_Stacked_Clipped.tif'

# training and validation as shape files
# training = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'
# validation = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'

training = r'/work/pi_cschweik_umass_edu/Ryan/OTH_Ground_Truth_Data/Removed or Edited/OTH_Poygons_withTrees_06Nov2023_4326.shp'
validation = r'/work/pi_cschweik_umass_edu/Ryan/OTH_Ground_Truth_Data/Removed or Edited/OTH_Poygons_withTrees_06Nov2023_4326.shp'

# what is the attributes name of your classes in the shape file (field name of the classes)?
attribute = 'ReClassV3'

# directory, where the classification image should be saved:
# classification_image = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.gtif'
classification_image = r'/work/pi_cschweik_umass_edu/Ryan/Classification_products/OTH_all_reclass_classification_v3_s2.gtif'

# directory, where the all meta results should be saved:
# results_txt = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.txt'
results_txt = r'/work/pi_cschweik_umass_edu/Ryan/Classification_products/OTH_all_reclass_classification_v3_s2.txt'

# laod training data and show all shape attributes

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

# extract the names of all attributes (fieldnames) in the shape file
attributes = []
ldefn = shape_layer.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    attributes.append(fdefn.name)
    
# print the attributes
print('Available attributes in the shape file are: {}'.format(attributes))

# prepare results text file:

print('Random Forest Classification', file=open(results_txt, "a"))
print('Processing: {}'.format(datetime.datetime.now()), file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))
print('PATHS:', file=open(results_txt, "a"))
print('Image: {}'.format(img_RS), file=open(results_txt, "a"))
print('Training shape: {}'.format(training) , file=open(results_txt, "a"))
print('Vaildation shape: {}'.format(validation) , file=open(results_txt, "a"))
print('      choosen attribute: {}'.format(attribute) , file=open(results_txt, "a"))
print('Classification image: {}'.format(classification_image) , file=open(results_txt, "a"))
print('Report text file: {}'.format(results_txt) , file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))

Available attributes in the shape file are: ['fid', 'id', 'left', 'top', 'right', 'bottom', 'Raw Subcla', 'Transect', 'PointNum', 'SubClass', 'Northing', 'Easting', 'Altitude', 'Notes', 'Class', 'Pre Angle', 'Post Angle', 'mu', 'Class mu', 'Pre/Post', 'Hydro', 'ReclassV2', 'layer', 'path', 'area', 'ReclassV3', 'ReClass v4']


In [6]:
# load image data
######THIS IS STEP THAT TAKES A LONG TIME#######

img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
print(img.shape, gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType), img.itemsize)
print(img_ds.GetRasterBand(1).ReadAsArray().itemsize)
for b in range(img.shape[2]):
    print(b)
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
row = img_ds.RasterYSize
col = img_ds.RasterXSize
band_number = img_ds.RasterCount

print('Image extent: {} x {} (row x col)'.format(row, col))
print('Number of Bands: {}'.format(band_number))


print('Image extent: {} x {} (row x col)'.format(row, col), file=open(results_txt, "a"))
print('Number of Bands: {}'.format(band_number), file=open(results_txt, "a"))
print('---------------------------------------', file=open(results_txt, "a"))
print('TRAINING', file=open(results_txt, "a"))
print('Number of Trees: {}'.format(est), file=open(results_txt, "a"))

(11908, 13206, 103) <class 'numpy.uint16'> 2
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
Image extent: 11908 x 13206 (row x col)
Number of Bands: 103


In [7]:
# laod training data from shape file
# img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

mem_drv = gdal.GetDriverByName('MEM')
mem_raster = mem_drv.Create('',img_ds.RasterXSize,img_ds.RasterYSize,1,gdal.GDT_UInt16)
#mem_raster.SetProjection(img_ds.GetProjection())
mem_raster.SetGeoTransform(img_ds.GetGeoTransform())
mem_band = mem_raster.GetRasterBand(1)
mem_band.Fill(0)
mem_band.SetNoDataValue(0)

att_ = 'ATTRIBUTE='+attribute
# http://gdal.org/gdal__alg_8h.html#adfe5e5d287d6c184aab03acbfa567cb1
# http://gis.stackexchange.com/questions/31568/gdal-rasterizelayer-doesnt-burn-all-polygons-to-raster
err = gdal.RasterizeLayer(mem_raster, [1], shape_layer, None, None, [1],  [att_,"ALL_TOUCHED=TRUE"])
assert err == gdal.CE_None

roi = mem_raster.ReadAsArray()

In [8]:
# # Display images
# plt.subplot(121)
# plt.imshow(img[:, :, 0], cmap=plt.cm.Greys_r)
# plt.title('RS image - first band')

# plt.subplot(122)
# plt.imshow(roi, cmap=plt.cm.Spectral)
# plt.title('Training Image')

# plt.show()

# Number of training pixels:
n_samples = (roi > 0).sum()
print('{n} training samples'.format(n=n_samples))
print('{n} training samples'.format(n=n_samples), file=open(results_txt, "a"))

# What are our classification labels?
labels = np.unique(roi[roi > 0])
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels), file=open(results_txt, "a"))

# Subset the image dataset with the training image = X
# Mask the classes on the training dataset = y
# These will have n_samples rows
X = img[roi > 0, :]
y = roi[roi > 0]

print('Our X matrix is sized: {sz}'.format(sz=X.shape))
print('Our y array is sized: {sz}'.format(sz=y.shape))

707591 training samples
training data include 6 classes: [  1   2   3  13  99 100]
Our X matrix is sized: (707591, 103)
Our y array is sized: (707591,)


In [9]:
rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=1, n_jobs=n_cores)

# verbose = 2 -> prints out every tree progression
# rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=2, n_jobs=n_cores)



X = np.nan_to_num(X)
rf2 = rf.fit(X, y)


# Save the trained model to a file
pickle_file = '/work/pi_cschweik_umass_edu/Ryan/Classification_products/OTH_all_reclass_classification_v3_s2.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(rf2, file)

print ('model pickled')

# With our Random Forest model fit, we can check out the "Out-of-Bag" (OOB) prediction score:

print('--------------------------------', file=open(results_txt, "a"))
print('TRAINING and RF Model Diagnostics:', file=open(results_txt, "a"))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100), file=open(results_txt, "a"))


# we can show the band importance:
bands = range(1,img_ds.RasterCount+1)

for b, imp in zip(bands, rf2.feature_importances_):
    print('Band {b} importance: {imp}'.format(b=b, imp=imp))
    print('Band {b} importance: {imp}'.format(b=b, imp=imp), file=open(results_txt, "a"))

    
# Let's look at a crosstabulation to see the class confusion. 
# To do so, we will import the Pandas library for some help:
# Setup a dataframe -- just like R
# Exception Handling because of possible Memory Error

try:
    df = pd.DataFrame()
    df['truth'] = y
    df['predict'] = rf.predict(X)

except MemoryError:
    print('Crosstab not available ')

else:
    # Cross-tabulate predictions
    print(pd.crosstab(df['truth'], df['predict'], margins=True))
    print(pd.crosstab(df['truth'], df['predict'], margins=True), file=open(results_txt, "a"))
    
# Predicting the rest of the image

# generate mask image from red band
mask = np.copy(img[:,:,0])
mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
old_shape = img.shape
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
# img = img[:, :, :int(img.shape[2])].reshape(new_shape)
img = img.reshape(new_shape)

print('Reshaped from {o} to {n}'.format(o=old_shape, n=img.shape))

[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:   19.1s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:  1.1min
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:  1.6min finished


model pickled
OOB prediction of accuracy is: 98.87124058954961%
Band 1 importance: 0.015130369447407348
Band 2 importance: 0.020097316811893674
Band 3 importance: 0.015252093113639438
Band 4 importance: 0.008606753216610868
Band 5 importance: 0.008920666620921546
Band 6 importance: 0.018016747847878777
Band 7 importance: 0.008723486866708537
Band 8 importance: 0.006001505014861539
Band 9 importance: 0.0038643595145276743
Band 10 importance: 0.026141171841620948
Band 11 importance: 0.009820692641197542
Band 12 importance: 0.004094610325419305
Band 13 importance: 0.010843167966968612
Band 14 importance: 0.0032552397679908225
Band 15 importance: 0.0049388029607229655
Band 16 importance: 0.003040646842026826
Band 17 importance: 0.005210452111392184
Band 18 importance: 0.0074523021634540715
Band 19 importance: 0.005325603419165448
Band 20 importance: 0.004171075285452449
Band 21 importance: 0.005060745171782924
Band 22 importance: 0.00553278582625066
Band 23 importance: 0.005234034941652306

[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    0.6s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:    2.0s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:    2.8s finished


predict       1    2       3    13     99     100     All
truth                                                    
1        160723    0       0     0      0       0  160723
2             0  936       0     0      0       0     936
3           990    0  123537     0      0       0  124527
13          300    0       0  7337      0       0    7637
99         2340    0       0     0  71316       0   73656
100        3417    0       0     0      0  336695  340112
All      167770  936  123537  7337  71316  336695  707591
Reshaped from (11908, 13206, 103) to (157257048, 103)


In [10]:
# img = np.nan_to_num(img)
img[np.isnan(img)] = 0.0
# class_prediction = rf.predict(img)

print ("DONE!")


DONE!


# **Apply Prediction**

In [11]:

slices = int(round(len(img)/20))

test = True

while test == True:
    try:
        class_preds = list()

        temp = rf.predict(img[0:slices+1,:])
        class_preds.append(temp)

        for i in range(slices,len(img),slices):
            print('{} %, derzeit: {}'.format((i*100)/(len(img)), i))
            temp = rf.predict(img[i+1:i+(slices+1),:])                
            class_preds.append(temp)

    except MemoryError as error:
        slices = slices/4
        print('Not enought RAM, new slices = {}'.format(slices))

    else:
        test = False
else:
    print('Class prediction was successful without slicing!')
#concatenate all slices and re-shape it to the original extend
try:
    class_prediction = np.concatenate(class_preds,axis = 0)
except NameError:
    print('No slicing was necessary!')
    
class_prediction = class_prediction.reshape(old_shape[:2])
print('Reshaped back to {}'.format(class_prediction.shape))


# # generate mask image from red band
# mask = np.copy(img[:,:,0])
# mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# plot mask

# plt.imshow(mask)

# mask classification an plot

class_prediction.astype(np.float16)
class_prediction_ = class_prediction*mask

cols = class_prediction.shape[1]
rows = class_prediction.shape[0]

class_prediction_.astype(np.float16)

driver = gdal.GetDriverByName("gtiff")
outdata = driver.Create(classification_image, cols, rows, 1, gdal.GDT_UInt16)
outdata.SetGeoTransform(img_ds.GetGeoTransform())##sets same geotransform as input
outdata.SetProjection(img_ds.GetProjection())##sets same projection as input
outdata.GetRasterBand(1).WriteArray(class_prediction_)
outdata.FlushCache() ##saves to disk!!
del outdata
print('Image saved to: {}'.format(classification_image))

[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    5.4s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:   16.5s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:   23.5s finished


4.999999745639381 %, derzeit: 7862852


[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    5.3s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:   16.7s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:   23.7s finished


9.999999491278762 %, derzeit: 15725704


[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    5.5s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:   17.0s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:   24.0s finished


...

89.99999542150887 %, derzeit: 141531336


[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    5.5s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:   16.7s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:   23.7s finished


94.99999516714824 %, derzeit: 149394188


[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    5.6s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:   17.2s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:   24.7s finished
[Parallel(n_jobs=60)]: Using backend ThreadingBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  80 tasks      | elapsed:    0.0s
[Parallel(n_jobs=60)]: Done 330 tasks      | elapsed:    0.0s
[Parallel(n_jobs=60)]: Done 500 out of 500 | elapsed:    0.0s finished


99.99999491278763 %, derzeit: 157257040
Class prediction was successful without slicing!
Reshaped back to (11908, 13206)
Image saved to: /work/pi_cschweik_umass_edu/Ryan/Classification_products/OTH_all_reclass_classification_v3_s2.gtif
